# Introduction to Monte Carlo Methods in Computational Science

## Overview

Monte Carlo methods are a powerful class of numerical techniques that leverage randomness to solve deterministic problems. These methods are widely used in physics, statistics, and computational science for tasks such as numerical integration, optimization, and probabilistic modeling.

In this notebook, we will explore the foundational concepts of Monte Carlo methods, including:

- Random number generation and how probability distributions play a role in sampling.
- Von Neumann rejection sampling, a technique for drawing random numbers from a complex probability distribution.
- The Metropolis algorithm, a Markov Chain Monte Carlo (MCMC) method for generating samples from an arbitrary probability distribution.
- Monte Carlo integration, a method for approximating integrals using random sampling.
- Importance sampling, an optimization technique that improves the efficiency of Monte Carlo integration by reducing variance.

## Why Monte Carlo Methods?

Many integrals and summations in physics and computational science are either high-dimensional or analytically intractable. Monte Carlo methods provide a way to approximate these computations using random sampling. The advantage of Monte Carlo approaches is their ability to handle high-dimensional spaces more effectively than traditional numerical quadrature methods.

## Roadmap

We will first examine how to generate random numbers from different distributions, then introduce the concept of rejection sampling as a method for drawing samples from non-trivial distributions. Building on this, we will explore the Metropolis algorithm, which extends these ideas into an iterative sampling technique. Finally, we will apply these methods to Monte Carlo integration, both with and without importance sampling, to illustrate how we can efficiently estimate integrals that are difficult to evaluate using traditional methods.

# Random Number Generation and Pseudorandom Number Generators

## Random Sequences
Computers are deterministic machines, making it impossible to generate truly random numbers. Instead, they use **pseudorandom number generators (PRNGs)**, which produce sequences of numbers that appear random but are generated by deterministic algorithms.

A sequence of numbers `{r_1, r_2, ... r_n}` is considered **random** if:
- There are no correlations between the numbers.
- The sequence follows a probability distribution $ P(r) $.
- It satisfies properties such as **unpredictability, independence, and absence of patterns**.

For example, a **uniformly distributed** sequence means that all numbers have an equal probability of appearing, represented as $ P(r) = 1 $ for a standard uniform distribution in $ [0,1] $.

## Pseudorandom Number Generators (PRNGs)
Since true randomness is not achievable in computers, PRNGs generate sequences using deterministic mathematical formulas. One common approach is the **Linear Congruential Generator (LCG)**, which follows the recurrence relation:

$$
r_{i+1} = (a r_i + c) \mod M
$$

where:
- $ r_1 $ is the **seed**, which initializes the sequence,
- $ a $, $ c $, and $ M $ are carefully chosen constants,
- The sequence has a periodicity of at most $ M $.

### Example of an LCG
Choosing $ c = 1 $, $ a = 4 $, $ M = 9 $, and $ r_1 = 3 $, the generated sequence cycles every 9 steps.

$$
\begin{aligned}
r_1 &= 3, \\
r_2 &= (4 \times 3 + 1) \mod 9 = 4, \\
r_3 &= (4 \times 4 + 1) \mod 9 = 8, \\
r_4 &= (4 \times 8 + 1) \mod 9 = 6, \\
&\quad 7, 2, 0, 1, 5, 3, \dots
\end{aligned}
$$

The period of an LCG depends on the choice of constants and follows conditions from the **Hull-Dobell Theorem**. The theorem states if and only if 3 conditions are met, will the period of the LCG be equal to $M$:
1. $m$ and $c$ are coprime
2. $a-1$ is divisible by all the prime factors of $M$
3. $a-1$ is divisible by 4 if m is divisible by 4

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

N = 100000
r = np.zeros(N+1)
M = 6537169170218971
a = 52361276312121

seed = 12391.
r[0] = seed
for i in range(1, N+1):
    r[i] = (a*r[i-1]) % M

r1 = np.zeros(N//2)
r2 = np.zeros(N//2)
for i in range(0,N,2):
    r1[i//2] = float(r[i])/float(M)
    r2[i//2] = float(r[i+1])/float(M)

plt.plot(r1, r2, 'o')

In [ ]:
normalized_r = r/float(M)
# histogram
plt.figure()
plt.hist(normalized_r, bins=100, density=True)
plt.show()

## Limitations of Simple PRNGs
- **Short cycles:** Poor choices of parameters can result in sequences repeating too quickly.
- **Correlations:** Some PRNGs, like the infamous **RANDU**, were flawed and led to incorrect scientific results.
- **Determinism:** Knowing one number can reveal the entire sequence.

To improve randomness, modern PRNGs use more sophisticated algorithms.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

N = 100000
r = np.zeros(N+1)
# This the infamous RANDU
M = 
a = 

seed = 12391.
r[0] = seed
for i in range(1, N+1):
    r[i] = (a*r[i-1]) % M

r1 = np.zeros(N//2)
r2 = np.zeros(N//2)
for i in range(0,N,2):
    r1[i//2] = float(r[i])/float(M)
    r2[i//2] = float(r[i+1])/float(M)

plt.plot(r1, r2, 'o')


In [ ]:
normalized_r = r/float(M)
# histogram
plt.figure()
plt.hist(normalized_r, bins=100, density=True)
plt.show()

In [ ]:
%matplotlib inline
r1 = np.zeros(int(N/3))
r2 = np.zeros(int(N/3))
r3 = np.zeros(int(N/3))

for i in range(0,N-1,3):
    r1[int(i/3)] = r[i]/float(M)
    r2[int(i/3)] = r[i+1]/float(M)
    r3[int(i/3)] = r[i+2]/float(M)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=20., azim=0)

ax.scatter(r1,r2,r3,marker="o"); 
plt.show()


## The Mersenne Twister
A widely used PRNG in scientific computing is the **Mersenne Twister (MT19937)**. It has several advantages:
- **Extremely long period** ($ 2^{19937}-1 $), preventing short cycles.
- **Good statistical properties**, passing most randomness tests.
- **Efficient generation**, making it ideal for Monte Carlo simulations.

Python’s `random` module and NumPy’s `numpy.random` module use variations of the Mersenne Twister, making it the standard for many Monte Carlo applications.


## Key Desirable Properties for PRNGs
For Monte Carlo simulations, an ideal PRNG should be:
- **Efficient**: Able to generate many numbers quickly.
- **Long-period**: Avoiding early repetition.
- **Repeatable**: For debugging purposes.
- **Resumable**: Allowing simulations to pick up where they left off.
- **Splittable**: Allowing independent parallel random streams.

# Non-uniform random distributions

In the previous section we learned how to generate random deviates with a uniform probability distribution in an interval $[a,b]$. This distributioon is normalized, so that $$\int _a^b {P(x)dx}=1.$$ Hence, $P(x)=1/(b-a)$.

Now, suppose that we generate a sequence $\{x_i\}$ and we take some function of it to generate $\{y(x_i)\}=\{y_i\}$. This new sequence is going to be distributed according to some probability density $P(y)$, such that $$P(y)dy=P(x)dx$$ or $$P(y)=P(x)\frac{dx}{dy}.$$

If we want to generate a desired normalized distribution $P(y)$, we need to solve the differential equation: 
$$\frac{dx}{dy}=P(y).$$ 
But the solution of this is $$x=\int _0^y {P(y')dy'}=F(y),$$ where $F(y)$ is known as the cumulative distribution function (CDF).

Therefore, 
$$y(x)=F^{-1}(x),$$ 
where $F^{-1}$ is the inverse of $F$.

### Exponential distribution

As an example, let us take $P(x)$ representing a uniform distribution in the interval $[0,1]$. We want to sample from
$$P(y)=\frac{dx}{dy}=e^{-y},$$  

To figure out the relevant transformation, we compute the CDF 
$$x=F(y)=\int _0^y {e^{-y'}dy'}=1-e^{-y}.$$
Solving for $y$ in terms of $x$, we have 
$$y=F^{-1}(x)=-\ln{(1-x)}.$$
This distribution occurs frequently in real problems such as the radioactive decay of nuclei. You can also see that the quantity $y/\lambda$ has the
distribution $\lambda e^{-\lambda y}$.


In [ ]:
%matplotlib inline 
import numpy as np
from matplotlib import pyplot as plt

N = 1000
x = np.random.random(N)

lam = 0.1
#Transformation equation
y = -np.log(1-x)/lam

binwidth=lam*5
plt.hist(y,bins=np.arange(0.,100.,binwidth),density=True)
plt.plot(np.arange(0.,100.,binwidth),lam*np.exp(-lam*np.arange(0.,100.,binwidth)))

You can get Gaussian random variables by the so-called Box-Muller transform. We'll demonstrate that it *works*, but for a thorough introduction read here: [Box-Muller Wikipedia](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform).

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

N=10000

def box_muller(u1, u2):
    z0 = np.sqrt(-2*np.log(u1))*np.cos(2*np.pi*u2)
    z1 = np.sqrt(-2*np.log(u1))*np.sin(2*np.pi*u2)
    return z0, z1

u1, u2 = np.random.rand(N), np.random.rand(N)

z0, z1 = box_muller(u1, u2)

fig, axs = plt.subplots(1,2, figsize=(12,5))

axs[0].hist(u1,bins=30,density=True)
axs[0].set_title('Uniform Distribution')
axs[0].set_xlim([0,1])

axs[1].hist(z0, bins=30, density=True, alpha=0.4, color='pink', edgecolor='black', label='z0')
axs[1].hist(z1, bins=30, density=True, alpha=0.4, color='lightgreen', edgecolor='black', label='z1')
axs[1].set_title('Normal Distribution')
axs[1].legend()

plt.tight_layout()
plt.show()

# von Neumann rejection sampling

Rejection sampling was invented by von Neumann in 1951. It is a simple and general method for generating arbitrary distributions. The idea is to sample uniformly from a region known to enclose the distribution of interest, and to reject any samples that lie outside the region.

To do rejection sampling, draw a plot with your probability distribution, and on the same graph, plot another curve $f(x)$ which has finite area and lies everywhere above your original distribution, that is for some constant $C$, $$P(x)\le C f(x).$$ We will call $f(x)$ the “comparison function”. You should be able to draw numbers from the comparison function $f(x)$. Generate a random number $x_i$ from $f(x)$ and a number $u$ form an uniform distribution. If $$u\le \frac{P(x)}{Cf(x)},$$ we accept $x_i$ as a valid sample; otherwise, we reject it and draw new random numbers.  The fraction of points accepted/rejected will depend on the ratio between the two areas. The closer the comparison function $f(x)$ resembles $P(x)$, the more points will be accepted. Ideally, for $P(x)=f(x)$, all the points will be accepted, and none rejected. However, in practice, this is not always possible, but we can try to pick $f(x)$ such that we minimize the fraction of rejected points.

No doubt you have seen rejection techniques in a common homework problem: computing $\pi$ by drawing random points in a square between $0$ and $1$ and counting how many were inside a circle. The circle is the region enclosing the distribution of interest, the uniform density on the circle, and the square is the region from which you can easily draw, as shown above. By rejection sampling, you computed the volume of the circle relative to the volume of the square. That ratio is $\pi/4$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def P(x):
    """Target distribution: Example - a custom probability density function (PDF)."""
    return 0.3 * np.exp(-0.2 * x**2) + 0.7 * np.exp(-0.5 * (x - 2)**2)

def f(x):
    """Proposal distribution: A Gaussian with mean 2 and standard deviation 2."""
    return np.exp(-0.5 * ((x - 2) / 2) ** 2)

# Constant C such that P(x) <= C * f(x) for all x
C = 1

def rejection_sampling(num_samples):
    """Von Neumann rejection sampling algorithm."""
    samples = []
    inter = 0
    while len(samples) < num_samples:
        # Sample x from proposal distribution f(x)
        x_candidate = np.random.normal(loc=2, scale=2)  # Sample from normal dist matching f(x)
        u = np.random.uniform(0, 1)  # Uniform random number in [0,1]
        inter=inter+1
        # Acceptance criterion
        if u < P(x_candidate) / (C * f(x_candidate)):
            samples.append(x_candidate)
        
    return np.array(samples), inter

# Generate samples
num_samples = 10000
samples, inter = rejection_sampling(num_samples)
print(inter)
# Plot results
x_vals = np.linspace(-5, 7, 1000)
plt.figure(figsize=(8, 5))
plt.hist(samples, bins=50, density=True, alpha=0.6, label='Sampled Distribution')
plt.plot(x_vals, P(x_vals), label='Target Distribution P(x)', linewidth=2)
plt.plot(x_vals, C * f(x_vals), linestyle='dashed', label='Scaled Proposal C*f(x)', linewidth=2)
plt.legend()
plt.xlabel('x')
plt.ylabel('Density')
plt.title('Von Neumann Rejection Sampling')
plt.show()

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

box_samples = np.random.rand(10000, 2)
plt.scatter(box_samples[:,0], box_samples[:,1])

# Random walk methods: the Metropolis algorithm

Suppose that we want to generate random variables according to an arbitrary probability density $P(x)$. The Metropolis algorithm produces a “random walk” of points $\{x_i\}$ whose asymptotic probability approaches $P(x)$ after a large number of steps. The random walk is defined by a “transition probability” $w(x_i \rightarrow x_j)$ for one value $x_i$ to another $x_j$ in order that the distribution of points $x_0$, $x_1$, $x_2$, ... converges to $P(x)$. In can be shown that it is sufficient (but not necessary) to satisfy the “detailed balance” condition 
$$p(x_i)w(x_i \rightarrow x_j) = p(x_j)w(x_j \rightarrow x_i).$$ 
This relation dos not specify $w(x_i \rightarrow x_j)$ uniquely. A simple choice is 
$$w(x_i \rightarrow x_j)=\min{\left[ 1,\frac{P(x_j)}{P(x_i)} \right] }.$$ 
This choice can be described by the following steps. Suppose that the “random walker” is a position $x_n$. To generate $x_{n+1}$ we

1.  choose a trial position $x_t=x_n+\delta _n$ , where the $\delta _n$ is a random number in the interval $[-\delta ,\delta]$.

2.  Calculate $w=P(x_t)/P(x_n)$.

3.  If $w \geq 1$ we accept the change and let $x_{n+1}=x_t$.

4.  If $w \leq 1$, generate a random number $r$.

5.  If $r \leq w$, accept the change and let $x_{n+1} = x_t$.

6.  If the trial change is not accepted, the let $x_{n+1}=x_n$.

It is necessary to sample a number of points of the random walk before the asymptotic probability $P(x)$ is attained. How do we choose the “step size” $\delta$? If $\delta$ is too large, only a small fraction of changes will be accepted and the sampling will be inefficient. If $\delta$ is too small, a large number will be accepted, but it would take too long to sample $P(x)$ over the whole interval of interest. Ideally, we want at least 1/3-1/2 of the trial steps to be accepted. We also want to choose $x_0$ such that the distribution $\{x_i\}$ converges to $P(x)$ as quickly as possible. An obvious choice is to begin the random walk at the point where $P(x)$ is maximum.

## The Gaussian distribution
Let's use the Metropolis algorithm to generate a Gaussian distribution 
$$P(x)=A \exp{(-x^2/2\sigma ^2)}.$$ 
Is the numerical value of the normalization constant $A$ relevant? Can we determine a reasonable choice of $\delta$ for a given $sigma$? (choose $x_0 = 0$.)
    

In [ ]:
def metropolis(xold):
    # Can we write a metropolis algorithm?
    return xold

In [ ]:
N = 100000

x = np.zeros(N)
delta = 3.
sigma = 20. 
sigma2 = sigma**2
#initial sample of points
xwalker = 20. 
Nwarmup = 5
for i in range(Nwarmup):
    xwalker = metropolis(xwalker)

subsample = True

x[0] = xwalker
for i in range(1,N):
    xold = x[i-1]
    if not subsample:
        xnew = metropolis(xold)
    else:
        for j in range(10):
            xnew = metropolis(xold)
            xold = xnew
    x[i] = xnew

binwidth = sigma/10
bins = np.arange(-50,50,binwidth)
plt.hist(x, bins=bins, density=True)

plt.rcParams['figure.dpi'] = 50
norm = 1./(sigma*np.sqrt(2*np.pi))
plt.plot(bins, norm*np.exp(-bins**2/(2*sigma2)), color='red')

# Monte Carlo integration

Imagine that we want to measure the area of a pond with arbitrary shape. Suppose that this pond is in the middle of a field with known area $A$. If we throw $N$ stones randomly, such that they land within the boundaries of the field, and we count the number of stones that fall in the pond $N_{in}$, the area of the pond will be approximately proportional to the fraction of stones that make a splash, multiplied by$A$: 
$$A_{pond}=\frac{N_{in}}{N}A.$$ 
This simple procedure is an example of the “Monte Carlo” method.

## Simple Monte Carlo integration

More generaly, imagine a rectangle of height $H$ in the integration interval $[a,b]$, such that the function $f(x)$ is within its boundaries. Compute $n$ pairs of random numbers $(x_i,y_i)$ such that they are uniformly distributed inside this rectangle. The fraction of points that fall within the area contained below $f(x)$, <span>*i.e.*</span>, that satisfy $y_i \leq f(x_i)$ is an estimate of the ratio of the integral of $f(x)$ and the area of the rectangle. Hence, the estimate of the integral will be given by:
$$\int _a^b{f(x)dx} \simeq I(N) = \frac{N_{in}}{N}H(b-a).$$

Another Monte Carlo procedure is based on the definition:
$$\langle f \rangle=\frac{1}{(b-a)} \int _a^b{f(x)dx}.$$ 
In order to determine this average, we sample the value of $f(x)$:
$$\langle f \rangle \simeq \frac{1}{N}\sum_{i=1}^{N}f(x_i),$$ 
where the $N$ values $x_i$ are distributed uniformly in the interval $[a,b]$. The
integral will be given by $$I(N)=(b-a) \langle f \rangle.$$

In [ ]:
#Simple integration
samples = np.random.random(100000)
x2 = samples**2
my_integral = np.mean(x2) # this is the integral of x^2 from 0 to 1
print(my_integral)

## Monte Carlo error analysis

The Monte Carlo method clearly yields approximate results. The accuracy depends on the number of values $N$ that we use for the average. A possible measure of the error is the “variance” $\sigma^2$ defined by:
$$\sigma ^2=\langle f^2 \rangle - \langle f \rangle ^2,$$ 
where
$$\langle f \rangle = \frac{1}{N} \sum_{i=1}^N f(x_i)$$ 
and
$$\langle f^2 \rangle = \frac{1}{N} \sum_{i=1}^{N} f(x_i)^2.$$ 
The “standard deviation” is $\sigma$. However, we should expect that the error decreases with the number of points $N$, and the quantity $\sigma$ defines above does not. Hence, this cannot be a good measure of the error.

Imagine that we perform several measurements of the integral, each of them yielding a result $I_n$. These values have been obtained with different sequences of $N$ random numbers. According to the central limit theorem, these values whould be normally dstributed around a mean $\langle I \rangle$. Suppouse that we have a set of $M$ of such measurements ${I_n}$. A convenient measure of the differences of these measurements is the “standard deviation of the means” $\sigma_M$:
$$\sigma_M ^2=\langle I^2 \rangle - \langle I \rangle ^2,$$ 
where
$$\langle I \rangle = \frac{1}{M} \sum_{n=1}^M I_n$$ 
and
$$\langle I^2 \rangle = \frac{1}{M} \sum_{n=1}^{M} I_n^2.$$ 
It can be proven that
$$\sigma_M \approx \sigma/\sqrt{N}.$$ 
This relation becomes exact in the limit of a very large number of measurements. Note that this expression implies that the error decreases with the square root of the number of trials, meaning that if we want to reduce the error by a factor 10, we need 100 times more points for the average.


## Variance reduction

If the function being integrated does not fluctuate too much in the interval of integration, and does not differ much from the average value, then the standard Monte Carlo mean-value method should work well with a reasonable number of points. Otherwise, we will find that the variance is very large, meaning that some points will make small contributions, while others will make large contributions to the integral. If this is the case, the algorithm will be very inefficient. The method can be improved by splitting the function $f(x)$ in two $f(x)=f_1(x)+f_2(x)$, such that the integral of $f_1(x)$ is known, and $f_2(x)$ as a small variance. The “variance reduction” technique, consists then in evaluating the integral of $f_2(x)$ to obtain:
$$\int _a^b{f(x)dx}=\int _a^b {f_1(x)dx} + \int _a^b{f_2(x)dx} = \int_a^b{f_1(x)dx}+J.$$

## Importance Sampling

Imagine that we want to sample the function $f(x)=e^{-x^2}$ in the interval $[0,1]$. It is evident that most of our points will fall in the region where the value of $f(x)$ is very small, and therefore we will need a large number of values to achieve a decent accuracy. A way to improve the measurement by reducing the variance is obtained by “importance sampling”. As the name says, the idea is to sample the regions with larger contributions to the integral. For this goal, we introduce a probability distribution $P(x)$ normalized in the interval of integration $$\int _a^b{P(x)dx} = 1.$$ Then, we can rewrite the integral of $f(x)$ as 
$$I=\int _a^b{\frac{f(x)}{P(x)}P(x)dx}$$ 
We can evaluate this integral, by sampling according to the probability distribution $P(x)$ and evaluating the sum
$$I(N)=\frac{1}{N} \sum_{i=1}^N \frac{f(x_i)}{P(x_i)}.$$ 
Note that for the uniform case $P(x)=1/(b-a)$, the expression reduces to the simple Monte Carlo integral.

We are free to choose $P(x)$ now. We wish to do it in a way to reduce and minimize the variance of the integrand $f(x)/P(x)$. The way to to this is picking a $P(x)$ that mimics $f(x)$ where $f(x)$ is large. If we are able to determine an appropiate $P(x)$, the integrand will be slowly
varying, and hence the variance will be reduced. Another consideration is that the generation of points according to the distribution $P(x)$
should be a simple task. As an example, let us consider again the integral 
$$I=\int _0^1 {e^{-x^2}dx}.$$ 
A reasonable choice for a weigh function is $P(x)=Ae^{-x}$, where $A$ is a normalization constant.

Notice that for $P(x)=f(x)$ the variance is zero! This is known as the zero variance property. There is a catch, though: The probability function $P(x)$ needs to be normalized, implying that in reality, $P(x)=f(x)/\int f(x)dx$, which assumes that we know in advance precisely the integral that we are trying to calculate!

In [ ]:
import numpy as np
import scipy.integrate as sci
def f(x):
    return np.exp(-x**2)

a, b = -20, 20
# use the trapezoid rule
def trapezoidal_rule(f, a, b, n=1000):
    x = np.linspace(a, b, n)
    y = f(x)
    dx = (b-a) / (n-1)
    return sci.trapezoid(y, dx = dx)

for n in [2**k for k in range(1,10)]:
    print(trapezoidal_rule(f, a, b, n=n))
# See how it does
trape = trapezoidal_rule(f, a, b, n=1000000)
print(trape)

In [ ]:
np.sqrt(np.pi)

In [ ]:
def monte_carlo_uniform(f,a,b,n=1000):
    samples = np.random.uniform(a,b,n)
    avg_value = np.mean(f(samples))
    return (b-a)*avg_value

def importance_sampling(f, n=1000):
    samples = np.random.normal(loc=0,scale=1,size=n)
    weights = f(samples) / (1/np.sqrt(2*np.pi)*np.exp(-samples**2/2))
    return np.mean(weights)
    
# trap_result = trapzoidal_rule(f, a, b)
importance_result = importance_sampling(f, n=1000000)
monte_carlo_result = monte_carlo_uniform(f, a, b, n=1000000)

print('Trapezoid:', trape)
print('Importance sampling:', importance_result)
print('Monte Carlo:', monte_carlo_result)

In [ ]:
imp, mc = [], []
ns = [10**k for k in range(3,8)]
for n in ns:
    imp.append(importance_sampling(f, n))
    mc.append(monte_carlo_uniform(f, a, b, n))

plt.scatter(ns, imp, label='Importance Sampling')
plt.scatter(ns, mc, label='Monte Carlo Uniform')
plt.axhline(truthbyn, color='red', label='Ground Truth')
plt.legend()
plt.xscale('log')
plt.yscale('log')


## Monte Carlo Integration in Quantum Field Theory

In quantum field theory, Feynman diagrams represent interactions between particles, and their corresponding integrals describe the probability amplitudes of such interactions. These integrals often take the form of loop integrals, which involve integrating over the momenta of virtual particles in intermediate states.

A basic example of a Feynman propagator integral in momentum space is:

$$
I = \int d^4k \frac{1}{(k^2 - m^2 + i\epsilon)}
$$

where:
- $ k $ is the four-momentum of a virtual particle,
- $ m $ is its mass,
- $ i\epsilon $ ensures proper contour deformation in complex analysis.

This type of integral is high-dimensional and does not have a simple closed-form solution. Traditional numerical integration methods struggle in such cases due to the curse of dimensionality, making Monte Carlo integration a powerful alternative.

In this section, we apply Monte Carlo integration to approximate this 4D Feynman propagator integral. This technique is widely used in lattice QCD, particle physics simulations, and cross-section calculations in collider experiments.


In [ ]:
import numpy as np
from scipy.integrate import quad, romberg

def monte_carlo_integration(f, dim, N=100000):
    """
    Monte Carlo integration for a high-dimensional integral.
    
    Parameters:
        f : function
            Function to integrate
        dim : int
            Number of dimensions
        N : int, optional
            Number of Monte Carlo samples (default is 100000)
    
    Returns:
        float: Approximated integral value
    """
    # Generate random points in dim-dimensional space in range [-1,1]
    points = np.random.uniform(-1, 1, (N, dim))
    
    # Evaluate function at those points
    func_values = np.apply_along_axis(f, 1, points)
    
    # Compute Monte Carlo estimate
    volume = (2**dim)  # Since limits are [-1,1]
    integral_estimate = volume * np.mean(func_values)
    
    return integral_estimate

# Define Feynman Diagram integral function
def feynman_integrand(k):
    """
    Example Feynman diagram integral in 4D:
    
    Integral of the form: 
    \int d^4k \frac{1}{(k^2 - m^2 + i\epsilon)}
    
    Assuming m = 1, epsilon = 1e-6 for numerical stability
    """
    m2 = 1.0  # Mass squared
    epsilon = 1e-6  # Small imaginary part
    k_squared = np.sum(k**2)  # Compute k^2
    return 1.0 / (k_squared - m2 + epsilon)

# Compute Feynman diagram integral in 4D using Monte Carlo
result_feynman = monte_carlo_integration(feynman_integrand, dim=4, N=1000000)
print(f"Monte Carlo approximation of Feynman integral (4D): {result_feynman}")